# Query Folding Safe Data Type Changes
> Simple solution to make data type changes query folding safe

- toc: true 
- badges: true
- comments: true
- categories: [powerquery,M, queryfolding]
- hide: false

## Query Folding

You can read about query folding in detail in [official documentation](https://docs.microsoft.com/en-us/power-query/power-query-folding) and the resources I have provided below. But at a high level, query folding allows us to create query steps that send a single native query back to the data source instead of the mashup engine doing the transformations. As much as possible, we want to design queries so they fold. Not all PowerQuery transformations are foldable. One of those transformations is changing data type.


In [8]:

%%html

<iframe title="vimeo-player" src="https://player.vimeo.com/video/714177719?h=466f2d1c1b" width="640" height="360" frameborder="0" allowfullscreen></iframe>

## Checking Query Folding


It is still debatable whether changing data types actually breaks query folding. My own tests using SQL Server, Postgres (BigQuery, Bit.io), OData, Synapse Analytics Serverless/Dedicated servers show that most data type changes don't actually break folding when you **check the server logs**. But my friend [Nikola recently pointed out that](https://twitter.com/DataMozart/status/1529875011313467393) it may not always be the case. There are three ways to check if a query is folding : 
- Right click the step in PQ and check if View Native Query option is greyed out. If it's greyed out, folding has stopped. Below query is foldable as the option is not greyed out. However, If this option is greyed out it doesn't necessarily mean that query folding isn't happening. But if it is not greyed out then we can be sure that query folding is taking place for sure. This method is similar to using *Value.Metadata()* to check query folding ([link](https://en.brunner.bi/post/new-ways-to-check-if-queries-fold-in-power-query)). 

![](https://docs.microsoft.com/en-us/power-query/media/power-query-folding/query-folding-example.png)




- Use Start Diagnostics in Power Query and check the detailed diagnostics report for the query sent to the data source. You can watch [my video](https://usergroup.tv/videos/query-folding-in-power-bi/) above to learn about this technique. This process is definitely a bit tedious. You can also use SQL Profiler in DAX Studio or SSMS to trace queries and identify the queries generated. 

- Check the server logs. This is the most straightforward and sure fire way. But not everyone has access to these logs.  

## Changing Data Type

So how do we make data type changes, query folding safe? Easy -  **Use Table.TransformColumns() instead of Table.TransformColumnTypes()** .I have verified this with all the foldable data source types and it has always worked. If it doesn't work in your scenario, please let me know. 
In the screenshot below, I am connected to serverless SQL pool in Synapse Analytics. I will be changing data types for three columns UnitPrice (decimal), UnitPriceDiscountPct (decimal), Is_Promotion (text).

![](https://raw.githubusercontent.com/pawarbi/blog/master/images/QFDC%201.PNG)

### Decimal to Percentage

Custom Transformation : 

In [11]:
#hide_input
'''Table.TransformColumns(Data, {{"UnitPriceDiscountPct",Number.From, Percentage.Type}})'''

'Table.TransformColumns(Data, {{"UnitPriceDiscountPct",Number.From, Percentage.Type}})'


![](https://raw.githubusercontent.com/pawarbi/blog/master/images/QFDCS2.PNG)

### Decimal to Currency

Custom Transformation :

In [13]:
#hide_input
'''Table.TransformColumns(Data, {{"UnitPrice",Number.From, Currency.Type}})'''

'Table.TransformColumns(Data, {{"UnitPrice",Number.From, Currency.Type}})'


![](https://raw.githubusercontent.com/pawarbi/blog/master/images/QFDCS3.PNG)

### Text to Logical

Custom Transformation :

In [16]:
#hide_input
'''Table.TransformColumns(Data, {{"Is_Promotion",Text.From, type logical}})'''

'Table.TransformColumns(Data, {{"Is_Promotion",Text.From, type logical}})'

![](https://raw.githubusercontent.com/pawarbi/blog/master/images/QFDC4.PNG)

You can use the exact same pattern to make any data type change query folding safe, including for text and date columns.


In [19]:
#hide_input
'''Table.TransformColumns(Data, {{"ColumnName",<..>.From, type <enter type>}})'''

'Table.TransformColumns(Data, {{"ColumnName",<..>.From, type <enter type>}})'

## Summary

Data type changes do not always stop query folding. You have to check the server logs to ensure if it's truly folding or not. But you can use above described method to always force query folding. This method works for SQL server, OData and Postgres SQL.  

## Resources

- Chris Webb's Blog: https://blog.crossjoin.co.uk/2021/02/21/query-folding-on-sql-queries-in-power-query-using-value-nativequery-and-enablefoldingtrue/

- My Power Query, Query Folding, Power BI Guru, Alex Powers's [30 Day Query Folding Challenge](https://www.youtube.com/watch?v=9sV3hIn8VTY)

- [My previous blog](https://pawarbi.github.io/blog/power%20bi/powerquery/queryfolding/m/optimization/2022/01/25/parameter-valuenativequery-query-folding-where-clause-in-powerbi.html)

- Nikola Ilic's blogs: Nikola has written and presented extensively on this topic
    - https://data-mozart.com/query-folding-tricks-lies-ultimate-performance-test/

    - https://data-mozart.com/what-is-a-query-folding-in-power-bi-and-why-should-i-care/